# Hugging Faceで公開されている大規模言語モデルをSageMakerにデプロイ


* 対象モデル
  
  matsuo-lab/weblab-10b-instruction-sft
  
  https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft


### SageMakerライブラリーのインストール

In [1]:
%pip install sagemaker --upgrade


     |████████████████████████████████| 868 kB 4.6 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.181.0-py2.py3-none-any.whl size=1184816 sha256=5c0e137c7a4e34acee87efc2f3c500cba71970c252a175dc522165d5a434c506
  Stored in directory: /home/vscode/.cache/pip/wheels/e5/62/d5/de3a132e0044a606f7f75e6cae3666b6f08402c717b997b154
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.180.0
    Uninstalling sagemaker-2.180.0:
      Successfully uninstalled sagemaker-2.180.0
Note: you may need to restart the kernel to use updated packages.


### インポート

In [2]:
import sagemaker
import boto3


### IAMロールの取得

In [3]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role_name = 'AmazonSageMaker-ExecutionRole-20230617T201891' # Role name with `AmazonSageMakerFullAccess` policy attached
	role = iam.get_role(RoleName=role_name)['Role']['Arn']


Couldn't call 'get_role' to get Role ARN from role name inspiron14 to get Role path.


### モデル名などのパラメーターを指定

In [4]:
model_id = 'matsuo-lab/weblab-10b-instruction-sft'
instance_type = 'ml.g5.12xlarge'


### SageMakerへデプロイ

In [5]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

image_uri = get_huggingface_llm_image_uri(
  backend='huggingface', # or lmi
  # region=region
)

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID': model_id, # model_id from hf.co/models
  'HF_TASK':'text-generation',          # NLP task you want to use for predictions
  # 'HF_MODEL_QUANTIZE':'bitsandbytes',
  'SM_NUM_GPUS': '2',
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
  env=hub,                            # configuration for loading model from Hub
  role=role,                          # IAM role with permissions to create an endpoint
  # transformers_version='4.28',        # Transformers version used
  # pytorch_version='2.0',              # PyTorch version used
  # py_version='py310',                 # Python version used
  image_uri=image_uri
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=600,
)


----------!

### 推論

In [6]:
text = "大規模言語モデルについて説明してください。"
text = f'以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{text}\n\n### 応答:'

data = {
   'inputs': text,
   'parameters': {
        'max_new_tokens': 100,
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.95  
   }
}

# request
result = predictor.predict(data)

result


[{'generated_text': '以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n大規模言語モデルについて説明してください。\n\n### 応答:大規模言語モデルは、大規模なデータセットから学習することができるニューラルネットワークモデルの一種です。大規模な言語モデルは、大量のデータセットを使用して、さまざまな種類の言語を認識し、学�'}]

### エンドポイントの削除

In [7]:
predictor.delete_endpoint(delete_endpoint_config=False)
predictor.delete_model()
